## NFL Headshots
Retreive Heashots for the Website

In [2]:
# !pip install nfl-data-py

In [167]:
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import nfl_data_py as nfl
from PIL import Image
import urllib

In [245]:
name_correction = {'Michael Pittman Jr.': 'Michael Pittman',
                    'AJ Dillon':'A.J. Dillon',
                    'Ken Walker III':'Kenneth Walker',
                    'Gabriel Davis':'Gabe Davis',
                    'Ronald Jones II':'Ronald Jones',
                    'Brian Robinson Jr.':'Brian Robinson',
                    'Cedrick Wilson Jr.':'Cedrick Wilson',
                    'Laviska Shenault Jr.':'Laviska Shenault',
                    'Gardner Minshew II':'Gardner Minshew',
                    'KJ Hamler':'K.J. Hamler',
                    'Terrace Marshall Jr.':'Terrace Marshall',
                    'Velus Jones Jr.':'Velus Jones',
                    'Pierre Strong Jr.':'Pierre Strong',
                    'Calvin Austin III':'Calvin Austin',
                    'Benny Snell Jr.':'Benny Snell',
                    'Anthony McFarland Jr.':'Anthony McFarland'
                  }

freeagents = ['Devontae Booker','David Johnson', 'Devonta Freeman','Emmanuel Sanders','Cole Beasley', 
             'Latavius Murray','Jared Cook','T.Y. Hilton','Jordan Howard','Odell Beckham']

### Free Agent  Antonio Brown could not be found in the 2022 or 2021 NFL Roster files 

nfl_name_df = pd.DataFrame(list(name_correction.items()), columns=['Player','name'])
fa = pd.DataFrame(freeagents, columns=['name'])
fa

,name
0,Devontae Booker
1,David Johnson
2,Devonta Freeman
3,Emmanuel Sanders
4,Cole Beasley
5,Latavius Murray
6,Jared Cook
7,T.Y. Hilton
8,Jordan Howard
9,Odell Beckham


In [228]:
nfl.see_pbp_cols()

Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       ...
       'out_of_bounds', 'home_opening_kickoff', 'qb_epa', 'xyac_epa',
       'xyac_mean_yardage', 'xyac_median_yardage', 'xyac_success', 'xyac_fd',
       'xpass', 'pass_oe'],
      dtype='object', length=372)

In [229]:
d = pd.read_csv('Resources/Draft.csv') 
d.drop(d[d['AVG'] == 0].index, inplace = True)
missing = d

In [230]:
pbp2021 = nfl.import_pbp_data(years=[2021], downcast=True, cache=False, alt_path=None)

2021 done.
Downcasting floats.


In [231]:
# load additional team data 
pbpclean = pbp2021
teams = nfl.import_team_desc()
players = nfl.import_rosters([2022])
freeAgents = nfl.import_rosters([2021])

In [215]:
teams.head(2)

,team_abbr,team_name,team_id,team_nick,team_conf,team_division,team_color,team_color2,team_color3,team_color4,team_logo_wikipedia,team_logo_espn,team_wordmark,team_conference_logo,team_league_logo
0,ARI,Arizona Cardinals,3800,Cardinals,NFC,NFC West,#97233F,#000000,#ffb612,#a5acaf,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500/ari.png,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...
1,ATL,Atlanta Falcons,200,Falcons,NFC,NFC South,#A71930,#000000,#a5acaf,#a30d2d,https://upload.wikimedia.org/wikipedia/en/thum...,https://a.espncdn.com/i/teamlogos/nfl/500/atl.png,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...


In [216]:
players.columns

Index(['season', 'team', 'position', 'depth_chart_position', 'jersey_number',
       'status', 'player_name', 'first_name', 'last_name', 'birth_date',
       'height', 'weight', 'college', 'player_id', 'espn_id', 'sportradar_id',
       'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id',
       'sleeper_id', 'years_exp', 'headshot_url', 'week', 'game_type',
       'status_description_abbr', 'football_name', 'esb_id', 'gsis_it_id',
       'smart_id', 'entry_year', 'rookie_year', 'draft_club', 'draft_number'],
      dtype='object')

In [217]:
players.head(2)

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,...,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number
0,2022,ARI,DB,FS,28.0,Active,Charles Washington,Charles,Washington,1993-03-10,...,REG,A01,Charles,WAS082938,43680,32005741-5308-2938-c497-7cfd2d22cbf6,2016,2016,None,NaN
1,2022,ARI,DB,CB,33.0,Active,Antonio Hamilton,Antonio,Hamilton,1993-01-24,...,REG,A01,Antonio,HAM184824,43986,32004841-4d18-4824-5017-25555a65166e,2016,2016,None,NaN


In [246]:
# Join with the roster table to get player ids
d1 = d.merge(players[["player_id", "player_name"]], left_on="Player", right_on="player_name", how='left')
d8 = nfl_name_df.merge(players[["player_id", "player_name"]], left_on="name", right_on="player_name", how='left')
d9 = fa.merge(freeAgents[["player_id", "player_name"]], left_on="name", right_on="player_name", how='left')
d9

,name,player_id,player_name
0,Devontae Booker,00-0032972,Devontae Booker
1,David Johnson,00-0032187,David Johnson
2,Devonta Freeman,00-0031285,Devonta Freeman
3,Emmanuel Sanders,00-0027685,Emmanuel Sanders
4,Cole Beasley,00-0029000,Cole Beasley
5,Latavius Murray,00-0030513,Latavius Murray
6,Jared Cook,00-0027061,Jared Cook
7,T.Y. Hilton,00-0029608,T.Y. Hilton
8,Jordan Howard,00-0032780,Jordan Howard
9,Odell Beckham,00-0031235,Odell Beckham


In [178]:
# Drop all rows from d1 where player not found in the 2022 Roster
d2 = d1[d1['player_id'].notna()]
d2 = d2.reset_index()

In [222]:
# Find players who did not match nfl_data by name 
no_match = d1[d1.isna().any(axis=1)]

In [182]:
# Get a list of players names that could not be matched and write to csv. 
no_match = d1[d1.isna().any(axis=1)]
FantxNFL = no_match['Player']
FantxNFL
FantxNFL.to_csv('Resources/FantxNFL.csv', index=False)

In [250]:
# Find how the NFL has the name recorded
showme = players[players['last_name'] == 'Beckham'] 
showme = freeAgents[freeAgents['last_name'] == 'Brown'] 
showme['player_id']
showme['player_name']
showme.sample(2)

,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,...,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number
744,2021,DAL,DB,CB,30.0,Active,Anthony Brown,Anthony,Brown,1993-12-15,...,WC,A01,Anthony,BRO266417,43478,32004252-4f26-6417-50b1-3b13a5b426b0,2016,2016.0,DAL,189.0
2931,2021,TEN,WR,WR,11.0,Active,A.J. Brown,Arthur,Brown,1997-06-30,...,DIV,A01,A.J.,BRO413223,47834,32004252-4f41-3223-e4c5-1e30dffa87f8,2019,2019.0,TEN,51.0


In [204]:
# Get headshots for active players with NFL matching names 
index = 0  
for player_id in d2["player_id"]:
    headshot_url = players[players["player_id"] == player_id].iloc[0].headshot_url
    print(headshot_url)
    if headshot_url:
        im = Image.open(urllib.request.urlopen(headshot_url))
        print(f" {index} Images/{d2.Player[index]}.png")
        im.save(f"Images/{d2.Player[index]}.png")
    index += 1

https://static.www.nfl.com/image/private/f_auto,q_auto/league/n7fj5fjblf6pdgjdtphr
 0 Images/Tom Brady.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/tojsgl1ttwjkvo6x7wgy
 1 Images/Cooper Kupp.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/vts6mscfg5le5zugxbtj
 2 Images/Jonathan Taylor.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/qknkhe4w6liovjubzmnh
 3 Images/Justin Herbert.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/pbl27kxsr5ulgxmvtvfn
 4 Images/Joe Burrow.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/tl0nhc7kyf4aumbyzs5b
 5 Images/Austin Ekeler.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/vs40h82nvqaqvyephwwu
 6 Images/Patrick Mahomes.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/ccgevhaduyc9llngcnlu
 7 Images/Deebo Samuel.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/zdstmm4sloqnichybgja
 8 Images/Justin Jefferson.png
htt

 74 Images/Daniel Jones.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/rmjuft9a2mmkka1steof
 75 Images/Sam Darnold.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/komdw34m3vpzadqftxzh
 76 Images/Carson Wentz.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/dnq0smzas3ifwi7sydu6
 77 Images/Jalen Hurts.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/fuutugtpr0nikfvjwk67
 78 Images/Adam Thielen.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/c7jaefxw5xiawscvnqc0
 79 Images/Darren Waller.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/toiqpnmbuxgfwr8wkw4r
 80 Images/Dallas Goedert.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/hol7dluumxi1cqxyagg3
 81 Images/DeVonta Smith.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/objv34ak0xhzjvixm5sr
 82 Images/Davis Mills.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/rvesbybzmlucxce3hkwf


 153 Images/Jameson Williams.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/bcecxpc9nciff80xhipd
 154 Images/David Njoku.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/pxrjunbln50toj2tmvbs
 155 Images/Julio Jones.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/qj5xzxtbrpp2el7iosgk
 156 Images/Rondale Moore.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/p44nhjopxr4mwncbdx5l
 157 Images/K.J. Osborn.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/mq0spwjm7whbnz2iut32
 158 Images/Michael Gallup.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/t4fgherebb4wc3mtyfrk
 159 Images/Chuba Hubbard.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/kvowcl8ebdanhiuiwcrp
 160 Images/D'Onta Foreman.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/ovlnn7uuj2335ublzb6o
 161 Images/Gerald Everett.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/e96l

 229 Images/Ricky Seals-Jones.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/cqlktgcvfqbt8j8w6kel
 230 Images/Jauan Jennings.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/pjp1aahvacigoh0sorpg
 231 Images/Taysom Hill.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/qkfbmmwufx3fmqd9cth9
 232 Images/Nick Westbrook-Ikhine.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/fdypdomxjrtx9zwybhnn
 233 Images/Tyler Badie.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/ppkdc6afdqqzy8qxzuvt
 234 Images/Abram Smith.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/bzetcp5jlpis9ueb1kfk
 235 Images/Olamide Zaccheaus.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/ytgijlfni4cyqs0zozhz
 236 Images/Josiah Deguara.png
None
https://static.www.nfl.com/image/private/f_auto,q_auto/league/xvtbawgh0kycl6caxrb5
 238 Images/Eno Benjamin.png
https://static.www.nfl.com/image/private/f_auto,q_

In [220]:
# Get headshots for active players with corrected NFL matching names 
index = 0  
for player_id in d8["player_id"]:
    headshot_url = players[players["player_id"] == player_id].iloc[0].headshot_url
    print(headshot_url)
    if headshot_url:
        im = Image.open(urllib.request.urlopen(headshot_url))
        print(f" {index} Images/{d8.Player[index]}.png")
        im.save(f"Images/{d8.Player[index]}.png")
    index += 1

https://static.www.nfl.com/image/private/f_auto,q_auto/league/mkhwsbbkzcjzp7hd8u5x
 0 Images/Michael Pittman Jr..png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/w270owegdowada3phhr2
 1 Images/AJ Dillon.png
None
https://static.www.nfl.com/image/private/f_auto,q_auto/league/gybdtwydpj0rggczumgg
 3 Images/Gabriel Davis.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/afztpm6htamuh4vb7dz9
 4 Images/Ronald Jones II.png
None
https://static.www.nfl.com/image/private/f_auto,q_auto/league/zvirmx1m9vycmfpfbf1w
 6 Images/Cedrick Wilson Jr..png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/q5wvq22iqxiw9lyjemxy
 7 Images/Laviska Shenault Jr..png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/y2wmde4s0fgnudjvl4m9
 8 Images/Gardner Minshew II.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/t3lqvv3d2tn6syqqtrat
 9 Images/KJ Hamler.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/azlfs3xbnfr3deksqvua
 1

In [247]:
# Get headshots for free agents players from prior year roster 
index = 0  
for player_id in d8["player_id"]:
    headshot_url = players[players["player_id"] == player_id].iloc[0].headshot_url
    print(headshot_url)
    if headshot_url:
        im = Image.open(urllib.request.urlopen(headshot_url))
        print(f" {index} Images/{d8.Player[index]}.png")
        im.save(f"Images/{d8.Player[index]}.png")
    index += 1

https://static.www.nfl.com/image/private/f_auto,q_auto/league/mkhwsbbkzcjzp7hd8u5x
 0 Images/Michael Pittman Jr..png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/w270owegdowada3phhr2
 1 Images/AJ Dillon.png
None
https://static.www.nfl.com/image/private/f_auto,q_auto/league/gybdtwydpj0rggczumgg
 3 Images/Gabriel Davis.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/afztpm6htamuh4vb7dz9
 4 Images/Ronald Jones II.png
None
https://static.www.nfl.com/image/private/f_auto,q_auto/league/zvirmx1m9vycmfpfbf1w
 6 Images/Cedrick Wilson Jr..png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/q5wvq22iqxiw9lyjemxy
 7 Images/Laviska Shenault Jr..png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/y2wmde4s0fgnudjvl4m9
 8 Images/Gardner Minshew II.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/t3lqvv3d2tn6syqqtrat
 9 Images/KJ Hamler.png
https://static.www.nfl.com/image/private/f_auto,q_auto/league/azlfs3xbnfr3deksqvua
 1